In [ ]:
# ENS Controls Extractor con traduzione EN
import pandas as pd
import re
from deep_translator import GoogleTranslator
import time

FILE_EXCEL = 'CCN-STIC-808-Verificación-del-cumplimiento-medidas_ANEXO.xlsx'
OUTPUT_CSV = 'SpanishENS.csv'

translator = GoogleTranslator(source='es', target='en')

def translate_text(text, max_retries=3):
    if pd.isna(text) or text == '':
        return ''
    text_str = str(text).strip()
    if len(text_str) == 0:
        return ''
    
    max_length = 4500
    if len(text_str) > max_length:
        if ' | ' in text_str:
            parts = text_str.split(' | ')
            translated_parts = [translate_text(p.strip(), max_retries) for p in parts if p.strip()]
            return ' | '.join([p for p in translated_parts if p])
        chunks = [text_str[i:i+max_length] for i in range(0, len(text_str), max_length)]
        translated_chunks = [translate_text(c, max_retries) for c in chunks]
        return ' '.join([c for c in translated_chunks if c])
    
    for attempt in range(max_retries):
        try:
            result = translator.translate(text_str)
            time.sleep(0.2)
            return result
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return text_str

def extract_controls(df, sheet_name):
    controls = []
    current_control = None
    current_title = None
    questions = []
    
    for idx, row in df.iterrows():
        col1 = str(row[1]).strip() if pd.notna(row[1]) else ''
        col2 = str(row[2]).strip() if pd.notna(row[2]) else ''
        col3 = str(row[3]).strip() if pd.notna(row[3]) else ''
        
        if re.match(r'^(Mp\.|Op\.|Org\.)[a-z]{2,3}\.\d+', col1):
            if current_control and (current_title or questions):
                controls.append({
                    'Codice': current_control,
                    'Titolo_ES': current_title,
                    'Domande_ES': ' | '.join(questions),
                    'Num_Domande': len(questions),
                    'Categoria': sheet_name
                })
            current_control = col1
            current_title = col3
            questions = []
        elif (col1 == '□' or col1 == '☐') and col2 and len(col2) > 15 and '?' in col2:
            questions.append(col2)
    
    if current_control and (current_title or questions):
        controls.append({
            'Codice': current_control,
            'Titolo_ES': current_title,
            'Domande_ES': ' | '.join(questions),
            'Num_Domande': len(questions),
            'Categoria': sheet_name
        })
    return controls

def assign_priority(code):
    if code.startswith('Org.') or code.startswith('Op.pl') or code.startswith('Op.ac'):
        return 'HIGH'
    elif code.startswith('Mp.if') or code.startswith('Mp.per'):
        return 'HIGH'
    elif code.startswith('Op.'):
        return 'MEDIUM'
    elif code.startswith('Mp.'):
        return 'MEDIUM'
    return 'LOW'

# ESTRAZIONE
print("🔍 Estrazione controlli...\n")
sheets = ['ANEXO II -MARCO ORGANIZATIVO', 'ANEXO II -MARCO OPERACIONAL', 'ANEXO II-MEDIDAS DE PROTECCIÓN']
all_controls = []
for sheet in sheets:
    df = pd.read_excel(FILE_EXCEL, sheet_name=sheet, header=None)
    controls = extract_controls(df, sheet)
    all_controls.extend(controls)
    print(f"✓ {sheet}: {len(controls)} controlli")

df_controls = pd.DataFrame(all_controls)
df_controls['Priorità'] = df_controls['Codice'].apply(assign_priority)

print(f"\n✅ Totale: {len(df_controls)} | HIGH: {sum(df_controls['Priorità'] == 'HIGH')} | MEDIUM: {sum(df_controls['Priorità'] == 'MEDIUM')}")

# TRADUZIONE
print(f"\n🌐 Traduzione in inglese (può richiedere alcuni minuti)...\n")
df_controls['Titolo_EN'] = df_controls['Titolo_ES'].apply(translate_text)
df_controls['Domande_EN'] = df_controls.apply(lambda row: translate_text(row['Domande_ES']) if row['Num_Domande'] > 0 else '', axis=1)

# SALVA
df_final = df_controls[['Codice', 'Priorità', 'Titolo_ES', 'Titolo_EN', 'Num_Domande', 'Domande_ES', 'Domande_EN', 'Categoria']]
priority_map = {'HIGH': 0, 'MEDIUM': 1, 'LOW': 2}
df_final['_s'] = df_final['Priorità'].map(priority_map)
df_final = df_final.sort_values(['_s', 'Codice']).drop('_s', axis=1)
df_final.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig', sep=';')

print(f"💾 Salvato: {OUTPUT_CSV}")
print(f"\n📋 Anteprima:")
display(df_final[['Codice', 'Priorità', 'Titolo_EN', 'Num_Domande']].head(10))

🔍 Estrazione controlli...

✓ ANEXO II -MARCO ORGANIZATIVO: 0 controlli
✓ ANEXO II -MARCO OPERACIONAL: 132 controlli
✓ ANEXO II-MEDIDAS DE PROTECCIÓN: 81 controlli

✅ Totale: 213 | HIGH: 75 | MEDIUM: 138

🌐 Traduzione in inglese (può richiedere alcuni minuti)...



/tmp/ipykernel_109017/1477337659.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['_s'] = df_final['Priorità'].map(priority_map)


OSError: Cannot save file into a non-existent directory: '../Schemes'

In [11]:
df_final.to_csv('SpanishENS.csv', index=False, encoding='utf-8-sig', sep=';')
